# Analyze loan approval decisions automated with IBM Automation Decision Services

## Build trust in your decision automation with IBM Cloud Pak for Business Automation
This notebook shows the exploitation of a dataset of loan approvals automated by IBM Cloud Pak for Business Automation. More precisely we applied <b>Automation Decision Services</b> to model a low code decision service and deploy it as an automation REST endpoint.
Form there we use a simulation microservice project called SimuLatte to play a set of requests, compute the decisions and apply an analytical notebook to get insights.

Each decision captures the request with all input parameters, the answer with all output parameters computed by the decision logic including business rules and ML models, and the trace. 
In the trace you find the list of the executed rules and ML models.

The dataset contains 5000 automated decisions.


## 1 - Read the decision dataset
This report post-processes the result of a simulation that wrote a dataset of automated decisions in a data lake or a local file system.
The first decisions found in the simulation dataset look like below:

In [17]:
import json

#!pip install requests
#import requests

!pip install pandas
import pandas as pd

!pip install urllib3
import urllib

from urllib.parse import urlencode
from urllib.request import Request, urlopen

from pathlib import Path
import json
import random
import datetime
import os
import sys
import requests
import pandas as pd
import numpy as np
import tqdm
import uuid
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected = True)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

!pip install pgeocode
import pgeocode

In [2]:
datasinkPath = "https://raw.githubusercontent.com/PierreFeillet/decisions-analytics/main/data/ADS/loanApprovalWithTasks/ads-22.0.1-loanapprovalwithtask-withtrace-5K.json" #startSimulation.json()["dataSink"]["uri"]
    
outputData = []
line = ""
for line in urllib.request.urlopen(datasinkPath):
    line = line.rstrip()
    jsonLine = dict()
    line_to_json = json.loads(line)
    jsonLine["request"] = line_to_json["request"]
    if(line_to_json["trace"]=="empty"): 
        jsonLine["trace"] = "empty" 
    else: 
        jsonLine["trace"] = line_to_json["trace"]
    jsonLine["response"] = line_to_json["response"]

    outputData.append(jsonLine)

datasink = pd.json_normalize(outputData)
#datasink = pd.json_normalize(outputData, ["taskInformation", "ruleInformation"], ["totalTasksNotExecuted", "shortname", ["tasksNotExecuted", "taskInformation"]])
datasink.head(5)

,request.loan.amount,request.loan.loanToValue,request.loan.numberOfMonthlyPayments,request.loan.startDate,request.borrower.firstName,request.borrower.lastName,request.borrower.zipCode,request.borrower.creditScore,request.borrower.birthDate,request.borrower.yearlyIncome,...,response.borrower.yearlyIncome,response.borrower.spouse,response.borrower.SSN.serialNumber,response.borrower.SSN.areaNumber,response.borrower.SSN.groupCode,response.__DecisionID__,response.messages,response.dataValidated,response.yearlyInterestRate,response.insurance
0,783440,0.7,162,2005-06-01T00:00:00Z,Tanya,Wilson,04250,736,1964-05-12T00:00:00Z,113243,...,113243,None,6789,191,45,260a9e68-b5ad-4da0-8ed4-8af631dc7410,"[Too big Debt/Income ratio, Very low risk loan...",True,0.082,NaN
1,1787385,0.7,183,2005-06-01T00:00:00Z,Jason,Contreras,85690,527,1915-05-12T00:00:00Z,224186,...,224186,None,6789,659,45,d5fd984a-7384-4ca9-9e5d-14cf26234d44,[The loan cannot exceed 1300000],False,0.000,NaN
2,1733494,0.7,75,2005-06-01T00:00:00Z,William,Lawrence,22377,460,1949-05-12T00:00:00Z,39954,...,39954,None,6789,599,45,2a930cf3-604a-4e1b-b44a-59b4a08a5a4b,[The loan cannot exceed 1300000],False,0.000,NaN
3,1350004,0.7,72,2005-06-01T00:00:00Z,Gary,Guerrero,43401,751,1949-05-12T00:00:00Z,219998,...,219998,None,6789,437,45,36eb221a-de1f-45cc-a0c8-674f2bf754e0,[The loan cannot exceed 1300000],False,0.000,NaN
4,1210944,0.7,160,2005-06-01T00:00:00Z,Deborah,Martinez,32684,684,1900-05-12T00:00:00Z,72470,...,72470,None,6789,696,45,2580a8e6-04df-4ec6-a518-1008b1ce9cb9,"[Too big Debt/Income ratio, Very low risk loan...",True,0.082,NaN


## 2 - Business Metrics

From each raw decision we extract or compute a set of metrics. The ratio of accepted versus rejected applications is a metric.
All raw input and output metrics are eligible to be metrics. 
Main metrics used in this report are:
- accepted/rejected status,
- loan amount,
- borrower's credit score,
- borrower's yearly income

In [3]:
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected = True)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Graph configs
approval_colors = ["#00CC96", "#FF6692"]
color_indices = [0, 1]

pie_chart_approval_loan_names = ['approved', 'rejected']
pie_chart_loan_amount_names = ['approved amount', 'rejected amount']
bidirectional_bar_chart_label_v2=['number of applications', 'sum of applications amounts']
bidirectional_bar_chart_label=['number of applications', 'sum of applications amounts']
bidirectional_bar_chart_label=['number of rejected applications', 'sum of rejected applications amounts', 'number of approved applications', 'sum of approved applications amounts']
radar_chart_labels=['number of rejected applications', 'sum of rejected applications amounts', 'number of approved applications', 'sum of approved applications amounts', 'ratio approved/requested']

In [4]:
########### Metrics #############
select_approved_decision = '`response.approved` == True'
select_rejected_decision = '`response.approved` == False'

get_decision_approval = 'response.approved'
get_decision_amount = 'response.loan.amount'
get_decision_borrower_zipcode = 'response.borrower.zipCode'
get_decision_borrower_yearly_income = 'response.borrower.yearlyIncome'
get_decision_borrower_credit_score = "response.borrower.creditScore"
get_decision_id = 'response.__DecisionID__'
get_decision_reimbursement_rate = 'response.yearlyInterestRate'
get_decision_monthly_repayment = 'response.monthlyRepayment'


## 3 - Business KPIs

Key Performance Indicators are aggregated counters to judge the decision set. 
Examples of basic KPIs for loan approval are:
- Sum of approved applications / total number of applications
- Sum of rejected applications / total number of applications
- Total of approved loan amounts
- Total of rejected loan amounts 

Only limits are the imagination and computational power of Python, Scala or R languages.

In [5]:
########### KPI #############
approved_loan = datasink.query(select_approved_decision)
rejected_loan = datasink.query(select_rejected_decision)

########### Approved/Rejected PRINT #############
print("Number of approved loan request = ", approved_loan.shape[0])
print("Number of rejected loan request = ", rejected_loan.shape[0])
print("Rate of approval = ", approved_loan.shape[0]/datasink.shape[0])

print("Total approved amount = ",datasink.query(select_approved_decision)[get_decision_amount].sum())
print("Total rejected amount = ",datasink.query(select_rejected_decision)[get_decision_amount].sum())
print("Total requested amount = ",datasink[get_decision_amount].sum())


Number of approved loan request =  936
Number of rejected loan request =  4064
Rate of approval =  0.1872
Total approved amount =  277070430
Total rejected amount =  4765681958
Total requested amount =  5042752388


## 3 - Visualization 
### 3.1 Approval vs Rejection 
We use donut representations to get an overview of the acceptation of the loans, in terms of number of applications and loan amounts.

In [6]:
########### settings ###########
donut_hole = 0.7

########### Approved/Rejected PIE CHART #############
trace_approval = go.Pie(labels = pie_chart_approval_loan_names,
                            values = [approved_loan.shape[0], rejected_loan.shape[0]],
                            hole = donut_hole)

########### Approved/Rejected Amount PIE CHART #############
trace_amount = go.Pie(values= [datasink.query(select_approved_decision)[get_decision_amount].sum(),
                                 datasink.query(select_rejected_decision)[get_decision_amount].sum()],
                            labels=pie_chart_loan_amount_names,
                            hole = donut_hole,
                            name="Loan amount")

fig = make_subplots(rows=1, cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    column_widths=[0.5, 0.5],
                    #vertical_spacing=0.001,
                    subplot_titles=("Approval", "Approved/rejected amount")
                    )

fig.add_trace(trace_approval, row=1, col=1)
fig.add_trace(trace_amount, row=1, col=2)

fig.update_traces(hoverinfo='label', textinfo='value', textfont_size=15, textposition="outside",
                  marker=dict(colors=approval_colors, line=dict(color='#FFFFFF', width=7)))
                
fig.update_annotations(yshift=20) # subplot tiltes

fig.update_layout(
    barmode='stack',
    plot_bgcolor = "white",
    #paper_bgcolor = 'lightgray',
    height=600,
    title={'text': f"'number of applications' and 'sum of applications amounts'",
          'x':0.5,
          'y':0.98,
          'xanchor': 'center',
          'font_size': 20
    },
    annotations= [
         {
            "font": {
               "size": 15
            },
            "showarrow": False,
            "text": "<b>Approval</b>",
            "x": 0.23,
            "y": 0.42
         },
         {
            "font": {
               "size": 15
            },
            "showarrow": False,
            "text": "<b>Total amount</b><br> approved/rejected",
            "x": 0.775,
            "y": 0.4
         }
      ],
      )

iplot(fig)

### 3.2 Represent the decisions in a yearly income, loand amount, and approval outcome space
We leverage a scatter component to represent all decisions. Each decision is colored in pink if refused or green if accepted. 
The circle radius depends on the amount. You can zoom in, select one decision, and get a tooltip with a summary of the decision.

We added 2 lines in the view:
- one horizontal line that matches a business rule that rejectsany application with a loan amount greater than 1 M.
- one diagonal that separates the population of never accepted application from one where accepted applications are concentrated. This observation is consistent with busines rules that enforced a maximum income/reimbursement ratio.


In [7]:
def array_normalize(values, max_value=1):
    normalized_values = []
    for value in values:
        normalized_values.append((value*max_value)/max(values))
    return normalized_values

def set_colors(values, true_color="#00CC96", false_color="#FF6692"):
    colors = []
    for value in values:
        if(value): 
            colors.append(true_color) 
        else: 
            colors.append(false_color) 
    return colors

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                 x=datasink[get_decision_borrower_yearly_income],
                 y=datasink[get_decision_amount],
                 mode='markers',
                 marker_size=array_normalize(datasink[get_decision_borrower_credit_score], 20), 
                 marker_color=set_colors(datasink[get_decision_approval]),
                 text=datasink[get_decision_approval], 
                 hovertext=datasink[get_decision_id],
                 hovertemplate="<b>Income:</b> %{x}<br><b>Loan amount:</b> %{y}<br><b>Approval:</b> %{text}<br><b>Id:</b> %{hovertext}<extra></extra>",
                 name='V1'
))


fig.add_trace(
    go.Scatter(
        x=np.array([0, max(datasink[get_decision_borrower_yearly_income])]),
        y=max(datasink.query(select_approved_decision)[get_decision_amount]/datasink.query(select_approved_decision)[get_decision_borrower_yearly_income])*np.array([0, max(datasink[get_decision_borrower_yearly_income])]),
        mode="lines",
        line=go.scatter.Line(color="#109618"),
        showlegend=False)
)
fig.add_trace(
    go.Scatter(
        x=np.array([0, max(datasink[get_decision_borrower_yearly_income])]),
        y=max(datasink.query(select_approved_decision)[get_decision_amount])*np.array([1, 1]),
        mode="lines",
        line=go.scatter.Line(color="#B82E2E"), ##B82E2E
        showlegend=False)
)


fig.update_xaxes(title_text='Yearly income')
fig.update_yaxes(title_text='Loan amount')

fig.update_layout(
    barmode='stack', 
    showlegend=False,
    title={'text': f"Behaviours according to income and loan amounts",
          'x':0.5,
          'xanchor': 'center'
    })

iplot(fig)

### 3.3 Visualize approvals on yearly income and credit scores

We use a scatter view to represent the decisions this time with yearly income and credit score. We observe 2 gradients that cumulate their effects:
- the higher yearly income is the more likelyhood to get the loan accepted 
- the higher is the credit score the more likelyhood to get the loan accepted 

In [9]:
############ SCATTER CHART #############
fig = go.Figure()
max_marker_size = 30
fig.add_trace(go.Scatter(
                 x=datasink[get_decision_borrower_yearly_income],
                 y=datasink[get_decision_borrower_credit_score],
                 mode='markers',
                 marker_size=array_normalize(datasink[get_decision_amount], max_marker_size), 
                 marker_color=set_colors(datasink[get_decision_approval]),
                 text=datasink[get_decision_approval], 
                 hovertemplate="<b>Income:</b> %{x}<br><b>creditScore:</b> %{y}<br><b>Approval:</b> %{text}<extra></extra>",
                 name='V1'
))

fig.update_xaxes(title_text='Yearly income')
fig.update_yaxes(title_text='Credit score')

fig.update_layout(
    barmode='stack', 
    showlegend=False,
    title={'text': f"creditScore vs Income",
          'x':0.5,
          'xanchor': 'center'
    })

iplot(fig)

### 3.4 Visualize Multiple Business KPIs

We use a radar (named too as Kiviat diagram or spyder) to represent the relative performances of the decision set.

In [10]:
fig = go.Figure()

#radar dimension normalization 
radar_max_value = 100

values = np.array([[datasink.query(select_rejected_decision).shape[0], 
                    datasink.query(select_rejected_decision)[get_decision_amount].sum(), 
                    datasink.query(select_approved_decision).shape[0], 
                    datasink.query(select_approved_decision)[get_decision_amount].sum(), 
                    datasink.query(select_approved_decision).shape[0]/(datasink.query(select_approved_decision).shape[0]+datasink.query(select_rejected_decision).shape[0])
                   ],
                   [
                    datasink.shape[0],
                    datasink[get_decision_amount].sum(),
                    datasink.shape[0],
                    datasink[get_decision_amount].sum(),
                    1
                   ]
                  ])
normed_values = (values*radar_max_value) / values.max(axis=0)

def radar_normalization(values, max_value):
    normalized_values = []
    for value in values:
        normalized_values.append((value*max_value)/max(values))
    return normalized_values

fig.add_trace(go.Scatterpolar(
      r=normed_values[0],
      theta=radar_chart_labels,
      marker_color='rgb(0,204,150)',
      fillcolor='rgba(0,204,150,0.4)',
      fill='toself',
      name='V1',
))

fig.update_layout(
  height=800,
  polar=dict(
    #bgcolor= 'rgb(239, 239, 232)',
    radialaxis=dict(
      visible=True,
      range=[0, radar_max_value],
      #linecolor='red',
      #gridcolor='red',
      #axiscolor='red',
      #tickcolor='red',
    )),
  #showlegend=False
)

fig.show()

### 3.5 Show the approval distributions

We use violon representations to show the distributions of the approved versus rejected applications projected on the loan amount dimension.
As a confirmation accepted applications are under the 1M limit and are concentrated on low amounts. 

In [11]:
subplot_titles = ['Approved', 'Rejected']
select_decision_approval_status = [select_approved_decision, select_rejected_decision]
get_decision_value = [get_decision_amount, get_decision_amount]
fig = go.Figure()
for idx, approval in enumerate(select_decision_approval_status):
    fig.add_trace(go.Violin(y=datasink.query(approval)[get_decision_amount],
                            name=subplot_titles[idx],
                            box_visible=True,
                            meanline_visible=True,
                            points='all'
                            ))

fig.update_yaxes(title_text='Loan amount')

fig.update_layout(
    barmode='stack', 
    showlegend=False,
    title={'text': f"Loan amount according to approval status",
          'x':0.5,
          'xanchor': 'center'
    })

fig.show()

### 3.6 - Take benefit of geographical tagging
We leverage the zipcode input parameter to locate the borrowers. It can help to detect if any correlation between the location (country, state) and the approval decision.

In [12]:
# Use a MapBox token allocated for SimuLatte demos

mapbox_access_token = "pk.eyJ1IjoicGZlaWxsZXQiLCJhIjoiY2w1aTdmeG14MDVlaTNkcGxpNjRmMGYwdyJ9.2eo04GLwcicGOiAqdDbVuA"

nomi = pgeocode.Nominatim('us')

df=nomi.query_postal_code(datasink[get_decision_borrower_zipcode].values)

fig = go.Figure(data=go.Scattermapbox(
        lon = df['longitude'],
        lat = df['latitude'],
        text = df['place_name'],
        mode = 'markers',
        marker=go.scattermapbox.Marker(
            size=array_normalize(datasink[get_decision_borrower_credit_score], 20),
            color=set_colors(datasink[get_decision_approval])
        ),
        ))

fig.update_layout(
    title='Loan requests localisation',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=df['latitude'].mean(),
            lon=df['longitude'].mean()
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig.show()

## 4 - Go inside the decisions
We exploit the trace in each of the decision to extract how their outcome was computed. We get the executed and we aggregate them to understand their coverage for this decision set.

In [13]:
def get_rules(record, tasksList=dict()):
    rules = []
    fromTasks = tasksList
    if record["recordType"]=="Rule":
        #delete index in fromTasks
        for index in list(fromTasks.keys())[::-1]:
            if int(index.replace(".", "")) != int(record['recordIndex'].replace(".", "")[0:len(index.replace(".", ""))]): 
                fromTasks.pop(index)

        fromTasks[record['recordIndex']] = record['properties']['name']
        record["from"] = {k:v for k,v in fromTasks.items()}
        rules.append(record)
    else:
        if "nestedRecords" in record["properties"]:
            fromTasks[record['recordIndex']] = record['properties']['name']
            for new_record in record["properties"]["nestedRecords"]:
                rules.extend(get_rules(new_record, fromTasks))
    
    return rules

In [14]:
rules = []
not_executed_rules = []
number_of_rules_per_decision = []

for number_of_decision in range(0,len(outputData)):
    nested_records = outputData[number_of_decision]['trace']['rootRecord']['properties']['nestedRecords']
    not_executed_rules.extend(outputData[number_of_decision]['trace']['rootRecord']['properties']['notExecutedTasks'])
    for record in nested_records:
        rules.extend(get_rules(record))

#print("RULES : ", rules)

In [15]:
rules_counter = dict()
for rule in rules:
    if rule['properties']['name'] in rules_counter:
        rules_counter[rule['properties']['name']] += 1
    else :
        rules_counter[rule['properties']['name']] = 1

rules_coverage = pd.DataFrame(rules_counter.items(), columns=['Rule', 'Count'])
rules_coverage['Percentage']= (rules_coverage['Count']*100)/datasink.shape[0]

rules_coverage

,Rule,Count,Percentage
0,Validation.borrower.check SSN area number,5000,100.00
1,Preparation.initial score,3109,62.18
2,Preparation.neverBankruptcy,3109,62.18
3,Preparation.rate,3109,62.18
4,Preparation.salary2score,3109,62.18
5,Computation.repayment,3109,62.18
6,Eligibility.check income,2173,43.46
7,Eligibility.grade,3109,62.18
8,Approval.approval,3109,62.18
9,Validation.loan.check amount,1785,35.70


### 4.1 Rule coverage bar graph
This view show all executed rules with its execution percentage.
100% means that the rule has been executed in all decisions.
The rule coverage empowers business users to check if the statistics of rule execution match their expectations and hypothesis when applied to the applications, and data in general.

In [16]:
fig = go.Figure(go.Bar(
            x=rules_coverage["Count"],
            y=rules_coverage["Rule"],
            text=[f'{item} %' for item in rules_coverage['Percentage']],
            textposition="outside",
            hovertemplate="<b>%{y}</b><br><b>Count:</b> %{x}<br><b>Percentage:</b> %{text}<extra></extra>",
            orientation='h'))

fig.update_xaxes(title_text='Number of decisions')
fig.update_yaxes(title_text='Rules')

fig.update_layout(
    title='RULES COVERAGE',
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=50, t=70, b=70),
    height=700,
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)
fig.show()

## Conclusion
This notebook shows a business report to analyze a volume of loan approval decisions. It leverages automated decisions performed by IBM Cloud Pak for Business Automation and more precisely Automation Decision services.
This report has been generated by the SimuLatte project that takes care of the simulation and the run of analytics report templates.

Tiémoko Dembele and Pierre Feillet are members of IBM France lab and developping actively the SimuLatte project to empower simulations for business automation.